In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#создаем датафреймы по событиям и сабмишнам
events_data_test = pd.read_csv('events_data_test.csv')
submissions_data_test = pd.read_csv('submission_data_test.csv')

In [4]:
#добавляем в сабмишн датасет столбцы где переводим таймстемп в дату и день
submissions_data_test['date'] = pd.to_datetime(submissions_data_test.timestamp, unit='s')
submissions_data_test['day'] = submissions_data_test.date.dt.date
submissions_data_test.head()

,step_id,timestamp,submission_status,user_id,date,day
0,31971,1526800961,wrong,24370,2018-05-20 07:22:41,2018-05-20
1,31971,1526800976,wrong,24370,2018-05-20 07:22:56,2018-05-20
2,31971,1526800993,wrong,24370,2018-05-20 07:23:13,2018-05-20
3,31971,1526801054,correct,24370,2018-05-20 07:24:14,2018-05-20
4,31972,1526800664,wrong,24370,2018-05-20 07:17:44,2018-05-20


In [5]:
#аналогично для ивентс_дата
events_data_test['date'] = pd.to_datetime(events_data_test.timestamp, unit='s')
events_data_test['day'] = events_data_test.date.dt.date
events_data_test.head()

,step_id,timestamp,action,user_id,date,day
0,30456,1526893787,viewed,24417,2018-05-21 09:09:47,2018-05-21
1,30456,1526893797,viewed,24417,2018-05-21 09:09:57,2018-05-21
2,30456,1526893954,viewed,24417,2018-05-21 09:12:34,2018-05-21
3,30456,1526895780,viewed,24417,2018-05-21 09:43:00,2018-05-21
4,30456,1526893787,discovered,24417,2018-05-21 09:09:47,2018-05-21


In [6]:
#Реструктурируем events_data_test. В результате имеем датафрейм с инфой
# о том сколько каждый пользователь сделал discovered	passed	started_attempt	viewed
users_events_data_test = events_data_test.pivot_table(index='user_id',
                        columns='action',
                        values='step_id',
                        aggfunc='count',
                       fill_value=0).reset_index()
users_events_data_test.head()

action,user_id,discovered,passed,started_attempt,viewed
0,4,1,1,0,1
1,6,1,1,0,1
2,10,2,2,0,6
3,12,11,9,4,14
4,13,70,70,35,105


In [7]:
#аналогично для сабмитов
users_scores_test = submissions_data_test.pivot_table(index='user_id',
                        columns='submission_status',
                        values='step_id',
                        aggfunc='count',
                       fill_value=0).reset_index()
users_scores_test

submission_status,user_id,correct,wrong
0,12,1,0
1,13,29,36
2,15,10,30
3,21,24,103
4,35,7,35
...,...,...,...
2798,26775,46,160
2799,26780,16,7
2800,26785,3,1
2801,26796,2,3


In [8]:
#создаем серию из events_data_test[['user_id', 'day', 'timestamp']]
#где оставляем уникальные сочетания user_id и day
#группируем по user_id и оставляем timestamp'ы последнего в дне события
#находим разницу между timestamp'ми. np.absolute на случай если разность между timestamp'ми будет отрицальная
#получаем np.array с разницой времени между событиями в каждый из дней
gap_data_test = events_data_test[['user_id', 'day', 'timestamp']].drop_duplicates(subset=['user_id', 'day']). \
    groupby('user_id')['timestamp'].apply(list). \
apply(np.diff).apply(np.absolute).values

In [9]:
#преобразуем в серию
gap_data_test = pd.Series(np.concatenate(gap_data_test, axis=0))

In [10]:
#преобразуем timestamp в дни
gap_data_test = gap_data_test / (24 * 60 * 60)

In [11]:
#В users_data_test хранится время последнего действия для каждого пользователя
users_data_test = events_data_test.groupby('user_id', as_index=False).agg({'timestamp' : 'max'}).rename(columns ={'timestamp' :
                                                                                                        'last_timestamp'})

In [12]:
#Наращиваем users_data_test путем добавления инфы о users_scores_test
users_data_test = users_data_test.merge(users_scores_test, on='user_id', how='outer')

In [13]:
users_data_test = users_data_test.fillna(0)

In [14]:
#Наращиваем users_data_test путем добавления инфы о users_events_data_test
users_data_test = users_data_test.merge(users_events_data_test, how='outer')

In [15]:
users_data_test

,user_id,last_timestamp,correct,wrong,discovered,passed,started_attempt,viewed
0,4,1529331707,0.0,0.0,1,1,0,1
1,6,1546889407,0.0,0.0,1,1,0,1
2,10,1534593770,0.0,0.0,2,2,0,6
3,12,1537348641,1.0,0.0,11,9,4,14
4,13,1533162170,29.0,36.0,70,70,35,105
...,...,...,...,...,...,...,...,...
6179,26791,1544075959,0.0,0.0,1,1,0,1
6180,26795,1545732383,0.0,0.0,1,1,0,1
6181,26796,1545813661,2.0,3.0,6,4,2,12
6182,26799,1536329484,2.0,0.0,6,6,2,6


In [16]:
#B user_min_time_test хранится timestamp первого действия
user_min_time_test = events_data_test.groupby('user_id', as_index=False). \
    agg({'timestamp' : 'min'}).rename({'timestamp' : 'min_timestamp'}, axis=1)
user_min_time_test

,user_id,min_timestamp
0,4,1529331707
1,6,1546889407
2,10,1534585756
3,12,1537348474
4,13,1533034354
...,...,...
6179,26791,1544075959
6180,26795,1545732383
6181,26796,1545813566
6182,26799,1536258073


In [17]:
#Добавляем инфу о времени первого действия
users_data_test = users_data_test.merge(user_min_time_test, how='outer')


In [18]:
#В user_time собираем сразу id и таймстемп события (Возможно не надо)
events_data_test['user_time'] = events_data_test.user_id.map(str) + '_' + events_data_test.timestamp.map(str)

In [19]:
events_data_test.head()

,step_id,timestamp,action,user_id,date,day,user_time
0,30456,1526893787,viewed,24417,2018-05-21 09:09:47,2018-05-21,24417_1526893787
1,30456,1526893797,viewed,24417,2018-05-21 09:09:57,2018-05-21,24417_1526893797
2,30456,1526893954,viewed,24417,2018-05-21 09:12:34,2018-05-21,24417_1526893954
3,30456,1526895780,viewed,24417,2018-05-21 09:43:00,2018-05-21,24417_1526895780
4,30456,1526893787,discovered,24417,2018-05-21 09:09:47,2018-05-21,24417_1526893787


In [20]:
#два дня по таймстемпу (используется в файле обучения, тут возможно не надо)
# пока делаю так, чтобы следить за датафреймами(Потом после заливки на гит надо баловаться)
learning_time_threshold = 2 * 24 * 60 *60
learning_time_threshold

172800

In [21]:
#время порога двух дней относительно первого события(возможно не надо) 
# пока делаю так, чтобы следить за датафреймами(Потом после заливки на гит надо баловаться)
user_learning_time_threshold_test = user_min_time_test.user_id.map(str) + '_' + (user_min_time_test.min_timestamp + learning_time_threshold).map(str)

In [22]:
user_learning_time_threshold_test.head()

0     4_1529504507
1     6_1547062207
2    10_1534758556
3    12_1537521274
4    13_1533207154
dtype: object

In [23]:
#добавляем в dataframe
# пока делаю так, чтобы следить за датафреймами(Потом после заливки на гит надо баловаться)
user_min_time_test['user_learning_time_threshold'] = user_learning_time_threshold_test

In [24]:
#наращиваем events_data_test
# пока делаю так, чтобы следить за датафреймами(Потом после заливки на гит надо баловаться)
# проверял фильтрация на 2 дня ничего не меняет
events_data_test = events_data_test.merge(user_min_time_test[['user_id', 'user_learning_time_threshold']], how='outer')

In [25]:
#Аналогично для submissions_data_test
submissions_data_test['user_time'] = submissions_data_test.user_id.map(str) + '_' + submissions_data_test.timestamp.map(str)
submissions_data_test = submissions_data_test.merge(user_min_time_test[['user_id', 'user_learning_time_threshold']], how='outer')

In [26]:
#формируем dataframe для теста
X_test = submissions_data_test.groupby('user_id').day.nunique().to_frame().reset_index() \
.rename(columns={'day' : 'days'})

In [27]:
#считаем сколько затраено степов
steps_tried_test = submissions_data_test.groupby('user_id').step_id.nunique().to_frame().reset_index() \
.rename(columns={'step_id': 'steps_tried'})

In [28]:
steps_tried_test.head()

,user_id,steps_tried
0,4,0
1,6,0
2,10,0
3,12,1
4,13,29


In [29]:
# наращиваем X_test
X_test = X_test.merge(steps_tried_test, on='user_id', how='outer')

In [30]:
# наращиваем X_test путем реструктуризации submissions_data_test
X_test = X_test.merge(submissions_data_test.pivot_table(index='user_id',
                        columns='submission_status',
                        values='step_id',
                        aggfunc='count',
                       fill_value=0).reset_index())

In [31]:
X_test['correct_ratio'] = X_test.correct / (X_test.correct + X_test.wrong)

In [32]:
# наращиваем X_test путем реструктуризации events_data_test
X_test = X_test.merge(events_data_test.pivot_table(index='user_id',
                        columns='action',
                        values='step_id',
                        aggfunc='count',
                       fill_value=0).reset_index()[['user_id', 'viewed']], how='outer')

In [33]:
X_test = X_test.fillna(0)

In [34]:
#ВОЗМОЖНО users_data нафиг не нужен. Давай после первого варика залитого на ГИТ поиграемся)

In [35]:
#user_id кидаем в индекс датафрейма, а потом его дропаем с датафрейма
X_test = X_test.set_index(X_test.user_id)
X_test = X_test.drop(['user_id'], axis = 1)

In [36]:
#X_test готов к предикту. В первом файле поиграй с моделями 
#Ориентируйся на максимальный roc_auc
#Cначала в первом файле обучи модели, и сохрани их как нибудь
#Потом просто во второй файл выгружаешь модели и считай)
#не забудь сделать датафрейм с предиктами и потом его в csv и выгружай на степик

In [37]:
X_test

,days,steps_tried,correct,wrong,correct_ratio,viewed
user_id,,,,,,
4,0.0,0.0,0.0,0.0,0.000000,1
6,0.0,0.0,0.0,0.0,0.000000,1
10,0.0,0.0,0.0,0.0,0.000000,6
12,1.0,1.0,1.0,0.0,1.000000,14
13,2.0,29.0,29.0,36.0,0.446154,105
...,...,...,...,...,...,...
26791,0.0,0.0,0.0,0.0,0.000000,1
26795,0.0,0.0,0.0,0.0,0.000000,1
26796,1.0,2.0,2.0,3.0,0.400000,12


In [38]:
# загружаем модель
import pickle
with open('trained_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [39]:
loaded_model

,n_estimators,140
,criterion,'gini'
,max_depth,8
,min_samples_split,50
,min_samples_leaf,54
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [40]:
# по модели предсказываем тот или иной результат
res = loaded_model.predict_proba(X_test)

In [41]:
# формируем итоговый датафрейм
result = pd.DataFrame({'user_id': X_test.index, 'is_gone': res[:,1]})

In [42]:
result

,user_id,is_gone
0,4,0.012046
1,6,0.012046
2,10,0.024981
3,12,0.059742
4,13,0.499427
...,...,...
6179,26791,0.012046
6180,26795,0.012046
6181,26796,0.027888
6182,26799,0.059326


In [43]:
#сохраняем в csv файл
result.to_csv('out_1.csv', index=False)  

In [44]:
X_test

,days,steps_tried,correct,wrong,correct_ratio,viewed
user_id,,,,,,
4,0.0,0.0,0.0,0.0,0.000000,1
6,0.0,0.0,0.0,0.0,0.000000,1
10,0.0,0.0,0.0,0.0,0.000000,6
12,1.0,1.0,1.0,0.0,1.000000,14
13,2.0,29.0,29.0,36.0,0.446154,105
...,...,...,...,...,...,...
26791,0.0,0.0,0.0,0.0,0.000000,1
26795,0.0,0.0,0.0,0.0,0.000000,1
26796,1.0,2.0,2.0,3.0,0.400000,12
